# Segmentation to Information Exploration
Let's explore what kind of useful information we can get from a semantic segmentation mask, assuming the mask is pretty accurate. For now, while we don't have a great segmentation model trained yet, we'll do this using manually annotated data.

In [ ]:
%load_ext autoreload
%autoreload 2

## Getting Data

Here's a nicely representative sample of images:

In [ ]:
import os
import cv2
seginput_path = os.path.abspath("../representatives/segmentation/seginput")
segmap_path = os.path.abspath("../arctic_images_original_2/segmaps")
images = []
for path in os.listdir(seginput_path):
    if path[-4:] != ".jpg": continue
    name = os.path.basename(path)
    images.append({"name": name, "seginput": cv2.imread(os.path.join(seginput_path, path)),
        "segmap": cv2.imread(os.path.join(segmap_path, name[:-4]+".png"), cv2.IMREAD_GRAYSCALE)})
    print(name)

Let's see what the images look like with their segmentation masks overlaid:

In [ ]:
import matplotlib.pyplot as plt
import math

def simple_composite(ax, image):
    ax.set_title(f"{image['name'][:9]}...{image['name'][-7:]}")
    ax.imshow(cv2.cvtColor(image["seginput"], cv2.COLOR_BGR2RGB))
    ax.imshow(image["segmap"], alpha=0.25, cmap="tab20", vmax=4, interpolation="none")

def plot_all(images, plot_fn, adjust_fn=None):
    cols = 3
    rows = math.ceil(len(images)/cols)
    size = 5
    fig,axs = plt.subplots(rows, cols, figsize=(rows*size, cols*size))
    if adjust_fn is not None: adjust_fn(fig)
    for ax in axs.ravel(): ax.axis("off")
    for ax,image in zip(axs.ravel(), images):
        plot_fn(ax, image)

plot_all(images, simple_composite, lambda fig: fig.subplots_adjust(hspace=-0.5))

From now on, we'll consider only the segmentation masks themselves:

In [ ]:
def plot_mask(ax, image, map_key="segmap"):
    ax.set_title(f"{image['name'][:9]}...{image['name'][-7:]}")
    ax.imshow(image[map_key], cmap="tab20", vmax=4, interpolation="none")

plot_all(images, plot_mask, lambda fig: fig.subplots_adjust(hspace=-0.5))

## Horizon Detection

When there is no ice on the horizon, horizon detection should be straightforward: just detect the water/sky edge and fit a line to it. We might not even have to use Hough line detection, we could just do a line of best fit. This does not solve the problem of distortion.

Sometimes, there's ice on the horizon but it's obvious that the ice doesn't rise much above the horizon. However, sometimes there is tall ice in the foreground that rises substantially above the horizon. If it's jagged, we could detect this and use a default horizon, but if it's smooth, it could be mistaken for the horizon. I'm not yet sure how to solve this problem automatically. For now, let's just detect the [ice and water] to sky edge.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv_experiments.cv_common as cc
from cv_experiments.labelme2fastai import label2value

print(label2value)

# Find the sky edge by searching down each column for the first pixel that is not sky.
# We can use searchsorted because we know that that all sky pixels in a column appear in a contiguous block
# at the top of the column (this assumption should be validated elsewhere)
def sky_edge(img):
    return np.apply_along_axis(np.searchsorted, 0, (img != label2value["sky"]), True)

# Find a horizon as the line of best fit of the sky_edge
def populate_horizons(map_key="segmap", edge_key="sky_edge", line_key="line"):
    for image in images:
        image[edge_key] = sky_edge(image[map_key])
        width = image[map_key].shape[1]
        image[line_key] = np.polynomial.Polynomial.fit(np.arange(width)-width//2, image[edge_key], 1).convert()

def plot_line(ax, image, map_key="segmap", line_key="line"):
    width = image[map_key].shape[1]
    intercept,slope = image[line_key]
    plot_mask(ax, image, map_key=map_key)
    ax.plot((0, width), (intercept-slope*width//2, intercept+slope*width//2), "r--", linewidth=2.5)

populate_horizons()
plot_all(images, plot_line, lambda fig: fig.subplots_adjust(hspace=-0.5))

### Undistortion
It is now clear that the camera distortion that some of the images suffer from will be an issue. Luckily, we have previously found a set of coefficients to perform undistortion; we just need to make sure interpolation doesn't mess things up. Let's undistort and try the horizon detection again.

In [ ]:
# cv2.undistort performs interpolation that will break masks. Here, we do one undistortion for each value in the
# mask and then combine things again. A faster solution would be to go to the functions that compose cv2.undistort
# and do those ourselves with no interpolation.
def undistort_mask(map_key="segmap", dest_key="und_segmap"):
    for image in images:
        # Produce one binary layer per mask value. This is not terrible when there are just a few mask values
        mask_stack = np.stack([image[map_key] == v for v in label2value.values()], axis=-1).astype(np.uint8)
        mask_stack = cc.undistort(mask_stack, "new")  # Undistort them all
        new_mask = np.argmax(mask_stack, axis=-1)  # Select the "truest" value for each pixel
        for i,v in enumerate(label2value.values()):  # Map back from layer indices to mask values
            new_mask[new_mask == i] = v
        image[dest_key] = new_mask.astype(np.uint8)

undistort_mask()
populate_horizons("und_segmap", "und_sky_edge", "und_line")
plot_all(images, lambda ax,img: plot_line(ax, img, map_key="und_segmap", line_key="und_line"), lambda fig: fig.subplots_adjust(hspace=-0.5))

That's better. Now let's rotate the images so that horizons are horizontal…

In [ ]:
def rotate_images(map_key="und_segmap", line_key="und_line", dest_key="rot_segmap", scale_key="rot_scale", height_key="orig_height"):
    for image in images:
        height,width = image[map_key].shape[:2]
        image[height_key] = height
        intercept,slope = image[line_key]
        center = (width//2, intercept)  # Rotate about the midpoint of the horizon line
        angle = np.arctan(slope)
        scale = np.cos(angle)  # Scale down so we don't erase information
        rot_mat = cv2.getRotationMatrix2D(center, angle*180/np.pi, scale)
        # Add some extra height so we don't erase information
        result = cv2.warpAffine(image[map_key], rot_mat, (width, height+width),
            flags=cv2.INTER_NEAREST, borderValue=label2value["none"])
        image[dest_key] = result
        image[scale_key] = scale
        image[height_key] = height

rotate_images()
plot_all(images, lambda ax,img: plot_mask(ax, img, map_key="rot_segmap"))

…adjust each column of pixels so the horizon line is perfectly flat (the thinking behind this step might bear revisiting)…

In [ ]:
# Shift a column up/down by a specific amount
def shift_col(col, amt):
    res = np.empty_like(col)
    if amt > 0:
        res[:amt] = label2value["none"]
        res[amt:] = col[:-amt]
    elif amt < 0:
        res[:amt] = col[-amt:]
        res[amt:] = label2value["none"]
    else:
        res[:] = col
    return res

# Shift each column of mat up/down by delta amount, where delta is an array of length equal to mat's width
def shift_mat(mat, delta):
    res = np.empty_like(mat)
    for i in range(mat.shape[1]):
        sc = shift_col(mat[:,i], delta[i])
        res[:,i] = sc
    return res

def adjust_columns(map_key="rot_segmap", line_key="und_line", dest_key="adj_segmap", height_key="orig_height"):
    for image in images:
        intercept,_ = image[line_key]
        intercept = int(intercept)
        # Search search_range pixels up and down of the intercept for the [ice and water] to sky edge
        search_range = int(0.05*image[height_key])
        new_edge = sky_edge(image[map_key][(intercept-search_range):(intercept+search_range), :])
        # # If not found, default to the intercept
        # new_edge[new_edge == intercept-search_range] = intercept
        # new_edge[new_edge == intercept+search_range] = intercept
        delta = search_range-new_edge
        image[dest_key] = shift_mat(image[map_key], delta)

adjust_columns()
plot_all(images, lambda ax,img: plot_mask(ax, img, map_key="adj_segmap"))

…and crop so that the horizon line forms the top edge of the image.

In [ ]:
def crop_horizon(map_key="adj_segmap", line_key="und_line", dest_key="crp_segmap"):
    for image in images:
        width = image[map_key].shape[1]
        # The greatest height possible is still greater than the original height, but it's smaller than it was before
        new_height = width
        intercept,_ = image[line_key]
        intercept = int(intercept)
        image[dest_key] = image[map_key][intercept:intercept+new_height,:]

crop_horizon()
plot_all(images, lambda ax,img: plot_mask(ax, img, map_key="crp_segmap"))

## A Shift in Perspective
Though we've done some undistorting, rotating, scaling, offsetting, and cropping, our images are still in a coordinate system based off the camera. The x-axis correlates to heading relative to the camera and the y-axis to distance, but these are not one-to-one. We now want to change the coordinate system so the axes represent the two dimensions of actual space parallel to the ocean surface. OpenCV provides a way to do this providing a list of four known point mappings.

Let:
 * _w_ be the original image width (pixels)
 * _h_ be the original image height (pixels)
 * _u_ be the horizontal field of view (angle)
 * _v_ be the vertical field of view (angle)
 * _d_ be the distance to the horizon (meters)
 * _l_ be the camera's height above the water (meters)
 * _s_ be the scaling factor from earlier (dimensionless, slightly less than 1).

A point _h_ pixels down from the horizon (which would typically be out of the image frame) is an angle of _v_ degrees down from the horizon. Thus it is _90deg-v_ degrees up from the vertical. Thus we have the distance from the boat to this point as _tan(90deg-v)*l_. Let this distance be _b_.

Let's define our real coordinate system such that the y-axis starts at the boat and extends through the center of the field of view and the x-axis starts at the boat and is perpendicular to the y-axis. Both axes are parallel with (and flush against) a plane representing the idealized sea surface. Then we have the mappings _(cameraX, cameraY) -> (realX, realY)_:
 * _(0, 0) -> (-sin(u/s/2)*d, d)_
 * _(w, 0) -> ( sin(u/s/2)*d, d)_
 * _(0, h) -> (-sin(u/s/2)*b, b)_
 * _(w, h) -> ( sin(u/s/2)*b, b)_

However, the coordinate system of the final image can't be (realX, realY) -- for one thing, image coordinates can't go negative. What we want is for the minimum realX coordinate to map to 0 in imageX, the maximum realX coordinate to map to whatever our output width is in imageX, d in realY to map to 0 in imageY, and 0 in realY to map to whatever our output height is in imageY. Let _p_, _q_ be the desired output width and height respectively. Then we have the following much simpler mappings _(cameraX, cameraY) -> (imageX, imageY)_:
 * _(0, 0) -> (0, 0)_
 * _(w, 0) -> (p, 0)_
 * _(0, h) -> (p*(1-b/d)/2, (d-b)/d*q)_
 * _(w, h) -> (p*(1+b/d)/2, (d-b)/d*q)_

We have _image -> real_ mappings:
 * p -> sin(u/s)*d
 * q -> d

so for everything to be on the same scale, we should pick a scale c in pixels per meter and set
 * p = d*c
 * q = sin(u/s) * d * c

We can use these to perform the perspective transformation. At certain places in here we have made simplifications (like using sine incorrectly), but let's just get an idea of where we are:

In [ ]:
camera_props = {
    "camera_height": 5.,
    "horiz_fov": 98.,
    "vert_fov": 54.5,
}
camera_props["horizon_distance"] = 3570.*np.sqrt(camera_props["camera_height"])
# TODO: we can improve the estimate of near_distance by taking into account roll, proxied by horizon intercept
camera_props["near_distance"] = np.tan(np.deg2rad(90-camera_props["vert_fov"]))*camera_props["camera_height"]

def unperspective(map_key="crp_segmap", scale_key="rot_scale", height_key="orig_height", dest_key="per_segmap"):
    for image in images:

        scale = image[scale_key]
        img_width = image[map_key].shape[1]
        img_height = image[height_key]

        # horiz_factor = np.sin(np.deg2rad(horiz_fov/scale/2))
        # points_camera_to_real = [
        #     ((0,         0,        ), (-horiz_factor*horizon_distance, horizon_distance)),
        #     ((img_width, 0,        ), ( horiz_factor*horizon_distance, horizon_distance)),
        #     ((0,         img_height), (-horiz_factor*horizon_distance, near_distance   )),
        #     ((img_width, img_height), ( horiz_factor*horizon_distance, near_distance   ))]
        
        pixels_per_meter = 0.1
        display_height = int(camera_props["horizon_distance"]*pixels_per_meter)
        display_width = int(np.sin(np.deg2rad(camera_props["horiz_fov"]/scale))*camera_props["horizon_distance"]*pixels_per_meter)
        near_factor = camera_props["near_distance"]/camera_props["horizon_distance"]*100 # Multiplying by scalar here to decrease the effect
        points_camera_to_image = [
            ((0,         0,        ), (0, 0)),
            ((img_width, 0,        ), (display_width, 0)),
            ((0,         img_height), (display_width*(1-near_factor)/2, (1-near_factor)*display_height)),
            ((img_width, img_height), (display_width*(1+near_factor)/2, (1-near_factor)*display_height))]
        
        src_points, dst_points = zip(*points_camera_to_image)
        mat = cv2.getPerspectiveTransform(np.float32(src_points), np.float32(dst_points))
        res = cv2.warpPerspective(image[map_key], mat, (display_width, display_height), flags=cv2.INTER_NEAREST, borderValue=label2value["none"])
        image[dest_key] = res

unperspective()
plot_all(images, lambda ax,img: plot_mask(ax, img, map_key="per_segmap"))

A few problems here:
 1. The distance dilation towards the horizon is so great that we have to reduce the warping by a large factor just to get something that doesn't look incredibly warped.
 2. This shows the horizon as a line, where really it should be the arc of a circle.
 3. Resolution is becoming an issue -- in particular, our lack of interpolation is causing some weird artifacts.

Responses:
 1. We should probably use a logarithmic scale in the final map, since we will always have much greater real-space resolution closer to the camera. We should probably also use polar coordinates.
 2. The theory at play here is a "homography," which relates the coordinate system in one plane to that in another. Essentially, it seems the horizon ought to appear very slightly curved in the original image, which should make it circular in the tranformed image. Perhaps we can approximate this using a bit of distortion.
 3. Once we lock down the transformation pipeline, we can explore either doing it all at once with one matrix or upsampling first. We can also investigate representing the image in other formats that allow for more substantial interpolation.

Let's address #2 first. What do we _want_ the transformed image to look like? Well, the horizon needs to be the arc of a circle spanning _u/s_ degrees. The sides and bottom should be lines, since lines in camera space map to lines in real space (this can be verified with a whiteboard and a phone camera). Assume for simplicity that pitch and roll are zero. Then we'd want something like this:

In [ ]:
def compose_template():
    pixels_per_meter = 2
    near_exaggeration = 1  # Factor by which to exaggerate the cutoff at the bottom so it's actually visible
    theta = camera_props["horiz_fov"]
    height = int(camera_props["horizon_distance"]*pixels_per_meter)
    width = int(2*np.sin(np.deg2rad(theta/2))*height)
    print(f"w={width}px, h={height}px")
    template = np.zeros((height,width))
    cv2.ellipse(template, (width//2, height), (height, height), -90, -theta/2, theta/2, 255, -1)  # Main arc
    near_px = int(camera_props["near_distance"]*pixels_per_meter*near_exaggeration)
    print(f"Near cutoff: {near_px}px")
    cv2.rectangle(template, (0, height-near_px), (width, height), 0, -1) # Cutoff at the bottom
    template_props = {
        "theta": theta,
        "pixels_per_meter": pixels_per_meter,
        "near_exaggeration": near_exaggeration,
        "near_px": near_px
    }
    return template, template_props

def plot_grayscale(img):
    plt.imshow(img, cmap="gray", vmin=0, vmax=255)
    plt.gca().axis("off")
    plt.show()

template, template_props = compose_template()
plot_grayscale(template)

Now let's do the homography in reverse:

In [ ]:
def draw_marker(img, x, y):
    radius = int(np.sqrt(np.prod(img.shape[:2]))*0.025)
    cv2.ellipse(img, (int(x), int(y)), (radius, radius), 0, 0, 360, 127, -1)

def untemplate(template, template_props, map_key="per_segmap"):
    height, width = template.shape[:2]
    horiz_down = height-np.tan(np.deg2rad(90-template_props["theta"]/2))*width/2
    bottom_side = np.tan(np.deg2rad(template_props["theta"]/2))*template_props["near_px"]

    dest_height, dest_width = images[0][map_key].shape[:2]
    dest_horiz_down = dest_height/10
    dest_close_up_deg = np.rad2deg(np.arctan(camera_props["near_distance"]*template_props["near_exaggeration"]/camera_props["camera_height"]))
    dest_close_down = dest_horiz_down+(dest_height/camera_props["vert_fov"])*(90-dest_close_up_deg)

    dest_border = 100

    points = [
        ((0, horiz_down), (dest_border, int(dest_horiz_down))),
        ((width, horiz_down), (dest_width-dest_border, int(dest_horiz_down))),
        ((width//2-bottom_side, height-template_props["near_px"]), (dest_border, int(dest_close_down))),
        ((width//2+bottom_side, height-template_props["near_px"]), (dest_height-dest_border, int(dest_close_down))),
    ]

    annotated = template.copy()
    for p,_ in points: draw_marker(annotated, *p)
    plot_grayscale(annotated)

    src_points, dst_points = zip(*points)    
    mat = cv2.getPerspectiveTransform(np.float32(src_points), np.float32(dst_points))
    res = cv2.warpPerspective(template, mat, (dest_width, dest_height), flags=cv2.INTER_NEAREST, borderValue=label2value["none"])
    return res

plot_grayscale(untemplate(template, template_props))

A few issues:
 1. The top of that output shape doesn't look at all curved. Theoretically it _is_, but not enough that we can reproduce it at sane resolution.
 2. The sides are very jagged. Again, theoretically, at infinite resolution, they'd be straight, but we're already working with extremely high resolution and they're quite jagged. This speaks to how much information we would be destroying in mapping to the arc-type image, which we'd have trouble getting back when mapping again to the log-polar image.

Thus it seems like we should stop trying to use homography matrices -- at least directly -- and just do our own transforms. We will continue using the undistortion code that make horizons straight lines even though technically they should be very slightly curved, then artificially correct for this later.

## A Shift in Perspective, Take Two

Our goal is to go from a camera image to a map where the y-axis is log(distance) and the x-axis is angle. Basically, given a coordinate (tr, lr) in scaled log-polar space (the destination), we need to define a function that specifies where it comes from in (xc, yc) in camera space (the source). We can do this by way of (xr, yr) real Cartesian space. The way we'll implement real Cartesian to camera image will still end up using a homography matrix, but since we'll end up combining this and the log-polar remapping into one transformation, we won't have the same issues.

In [ ]:
logb = np.log10
expb = lambda a: 10**a

def y2dist(y, height, min_distance, max_distance):
    return expb((height-y)*(logb(max_distance/min_distance)/height)+logb(min_distance))
def dist2y(dist, height, min_distance, max_distance):
    return height-logb(dist/min_distance)/logb(max_distance/min_distance)*height

def log_polar_to_real_cartesian(ts, ls, width, height, t_range, min_distance=1, max_distance=10000):
    # tr: the x-coordinate in the log-polar image (px); could be a numpy vector
    # lr: the y-coordinate in the log-polar image (px); could be a numpy vector
    # width: the log-polar image width (px)
    # height: the log-polar image height (px)
    # t_range: the angle spanned by the x-axis (deg)
    # min_distance: the distance from the boat at the bottom of the image (m)
    # max_distance: the distance from the boat at the top of the image (m)

    try: len_tr = len(ts)
    except TypeError: len_tr = None
    try: len_lr = len(ls)
    except TypeError: len_lr = None
    if not len_tr == len_lr: raise ValueError(f"Lengths of tr and lr must be the same; got {len_tr} and {len_lr}")
    
    distance = y2dist(ls, height, min_distance, max_distance)
    theta = ts*(t_range/width)-(t_range/2)

    xr = np.sin(np.deg2rad(theta))*distance
    yr = np.cos(np.deg2rad(theta))*distance
    return xr, yr

# Check distance:
print(log_polar_to_real_cartesian(np.array([75, 75, 75]), np.array([100, 50, 0]), 150, 100, 90))
# Check theta:
print(log_polar_to_real_cartesian(np.array([0, 75, 150]), np.array([50, 50, 50]), 150, 100, 90))
# Check scalar:
print(log_polar_to_real_cartesian(0, 50, 150, 100, 90))

In [ ]:
# TODO: all the properties we need to do this math here are the same for every image -- except scale. For performance,
# we could factor that out, calculate one remapping matrix for all the images, then rescale per image later.
def real_cartesian_to_camera(xr, yr, width, height, scale, camera_props=camera_props):
    # xr: the x coordinate in real space (m); could be a numpy vector
    # yr: the y coordinate in real space (m); could be a numpy vector
    # width: the width of the camera image (px)
    # height: the height of the camera image (px)
    # camera_props: a dictionary of necessary camera properties
    # scale: the per-image scaling factor from earlier (dimensionless, slightly less than 1)

    horizon_distance = camera_props["horizon_distance"]
    near_distance = camera_props["near_distance"]
    spread = np.tan(np.deg2rad(camera_props["horiz_fov"]/scale/2))
    points_real_to_camera = [
        ((-spread*horizon_distance, horizon_distance), (0,     0,    )),
        (( spread*horizon_distance, horizon_distance), (width, 0,    )),
        ((-spread*near_distance,    near_distance   ), (0,     height)),
        (( spread*near_distance,    near_distance   ), (width, height)),
    ]
    src_points, dst_points = zip(*points_real_to_camera)
    mat = cv2.getPerspectiveTransform(np.float32(src_points), np.float32(dst_points))

    # cv2.perspectiveTransform needs a very specific shape
    points = np.stack([xr, yr], -1)[np.newaxis].astype(np.float32)
    scalar_input = (points.ndim == 2)
    if scalar_input: points = points[np.newaxis]
    transformed = cv2.perspectiveTransform(points, mat)
    xc, yc = transformed[0, :, 0], transformed[0, :, 1]
    if scalar_input: xc, yc = xc[0], yc[0]
    return xc, yc

# Check corners:
print(tuple(map(np.int32, real_cartesian_to_camera(
    np.array([-2000, 2000, -2, 2]), np.array([6000, 6000, 5, 5]), 150, 100, 1))))
# Check scalar:
print(real_cartesian_to_camera(-2000, 6000, 150, 100, 1))

In [ ]:
output_props = {
    "width": 500,
    "height": 279,
    "t_range": camera_props["horiz_fov"]*1.1,  # A bit of extra space for if horizons were diagonal
    "min_distance": 3,
    "max_distance": 10000
}

def camera_to_log_polar(final_width=output_props["width"], final_height=output_props["height"], t_range=output_props["t_range"], min_distance=output_props["min_distance"], max_distance=output_props["max_distance"], camera_props=camera_props, map_key="crp_segmap", scale_key="rot_scale", height_key="orig_height", dest_key="log_segmap"):
    for image in images:
        camera_width = image[map_key].shape[1]
        camera_height = image[height_key]
        x_in = np.tile(np.arange(final_width), (final_height, 1)).astype(np.float32).reshape(-1)
        y_in = np.tile(np.arange(final_height), (final_width, 1)).T.astype(np.float32).reshape(-1)
        x_inter, y_inter = log_polar_to_real_cartesian(x_in, y_in, final_width, final_height, t_range, min_distance, max_distance)
        x_out, y_out = real_cartesian_to_camera(x_inter, y_inter, camera_width, camera_height, image[scale_key], camera_props)
        y_out[y_out < 0] = np.nan  # Don't extend beyond the horizon
        x_out = x_out.reshape((final_height, final_width))
        y_out = y_out.reshape((final_height, final_width))
        horizon_height = dist2y(camera_props["horizon_distance"], final_height, min_distance, max_distance)
        y_out[:int(np.ceil(horizon_height)), :] = np.nan  # Artificially create a circular horizon rather than a line. In practice, the changes due to this are always well under a pixel in the original image (that's why the horizon appears to be a line in the first place), so this is more or less cosmetic and is not manipulation.
        result = cv2.remap(image[map_key], x_out, y_out, cv2.INTER_NEAREST, borderValue=label2value["none"])
        image[dest_key] = result

camera_to_log_polar()
plot_all(images, lambda ax,img: plot_mask(ax, img, map_key="log_segmap"), lambda fig: fig.subplots_adjust(hspace=-0.5))

Yes!!! That looks like what we want! Let's put a scale on it:

In [ ]:
def log_minor_ticks(d0, d1):  # Generate the minor ticks in a log plot using an outer product!
    exprange = expb(np.arange(int(np.floor(logb(d0))), int(np.ceil(logb(d1)))+1))
    all_ticks = (np.outer(exprange, np.arange(2, expb(1)))).ravel()
    return all_ticks[(d0 <= all_ticks) & (all_ticks <= d1)]

def x2t(x, width, t_range):
    return x*t_range/width-t_range/2
def t2x(t, width, t_range):
    return t*width/t_range+width/2

def plot_log_polar(ax: plt.Axes, image, map_key="log_segmap"):
    plot_mask(ax, image, map_key)

    ax.axis("on")
    # It's possible we could use pcolormesh and some more transformation matrices to get matplotlib to do log scales
    # for us… but that would basically be repeating work from above. I'd rather just construct the axes manually:
    d0, d1, t_range = output_props["min_distance"], output_props["max_distance"], output_props["t_range"]
    height, width = image[map_key].shape[:2]
    major_distances = expb(np.arange(np.ceil(logb(d0)), np.floor(logb(d1))+1)).astype(int)
    ax.set_yticks(dist2y(major_distances, height, d0, d1))
    ax.set_yticklabels(major_distances)
    ax.set_yticks(dist2y(log_minor_ticks(d0, d1), height, d0, d1), minor=True)
    major_angles = np.arange(0, int(np.floor(t_range/2))+1, 10)
    major_angles = np.concatenate([-major_angles[::-1], major_angles[1:]])
    ax.set_xticks(t2x(major_angles, width, t_range))
    # ax.set_xticklabels(np.vectorize(lambda x: f"{x:.1f}")(major_angles))  # Format floating-point labels
    ax.set_xticklabels(major_angles)

    ax.set_ylabel("Distance (m)")
    ax.set_xlabel("Angle (deg)")

plot_all(images, plot_log_polar, lambda fig: fig.subplots_adjust(hspace=-0.5, wspace=0.4))

Success! Next step is to handle interpolation better throughout all steps to get rid of that fringe effect at large distances, but the essential problems are solved.